In [115]:
import base64
import numpy as np
from watson_developer_cloud import NaturalLanguageClassifierV1
from os import listdir
from os.path import isfile, join
import csv
import re
from collections import *
import operator
import json

In [2]:
train_data = np.load("githubtrainingdata.npy")
test_data = np.load("githubtestdata.npy")


In [4]:
pls = {}
for row in range(len(train_data)):
    if train_data[row][1] not in pls:
        pls[train_data[row][1]] = []
    pls[train_data[row][1]].append(train_data[row][0])
    

In [93]:
chunk = 1024
with open('trainingdatanew.csv', 'w') as csvfile:
    fieldnames = ['text', 'pl']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    for i in train_data:
        for j in range(0,len(i[0]),chunk):
            text = re.sub(' +',' '," ".join(re.split(r'[^\w]', re.sub(re.compile("/\*.*?\*/",re.DOTALL ) ,"" ,i[0][j:j+chunk]))))   
            writer.writerow({'text': text, 'pl': i[1]})

In [10]:
def bayes_train(pldict, samples):
    plprobs = {}
    counts = Counter()
    for i in pldict.iterkeys():
        plprobs[i] = float(len(pldict[i]))/samples
        
    plwordprobs = {}
    plwordcounts = {}
    for pl in pldict.iterkeys():
        plwordprobs[pl] = {}
        plwordcounts[pl] = 0
    
    for pl in pldict.iterkeys():
        for i in pldict[pl]:
            counts.update(filter(None, re.split(r'[^\w]', re.sub(re.compile("/\*.*?\*/",re.DOTALL ) ,"" ,i))))
            for word in counts.iterkeys():
                if word not in plwordprobs[pl]:
                    plwordprobs[pl][word] = counts[word]
                else:
                    plwordprobs[pl][word] += counts[word]
                plwordcounts[pl] += counts[word]
            plwordcount = 0
            counts = Counter()
    for pl in plwordprobs.iterkeys():   
        for word in plwordprobs[pl]:
            plwordprobs[pl][word] = float(plwordprobs[pl][word])/plwordcounts[pl]
        
    
    return plprobs, plwordprobs
    
plrobs, plwordprobs = bayes_train(pls, len(training_data))

In [153]:
sorted(plwordprobs['sh'].iteritems(), key=operator.itemgetter(1) ,reverse=True)[:10]

[('echo', 0.03864587716931944),
 ('the', 0.03778674579624146),
 ('0', 0.033368860713964506),
 ('1', 0.030890171747144934),
 ('if', 0.02780092617570351),
 ('for', 0.027205853790126674),
 ('import', 0.02685395198273526),
 ('to', 0.02617263735275594),
 ('in', 0.02470367772682303),
 ('return', 0.023925887959715765)]

In [161]:
plprobs

{'go': 0.04184782608695652,
 'java': 0.2184782608695652,
 'js': 0.20597826086956522,
 'm': 0.06956521739130435,
 'py': 0.07880434782608696,
 'sh': 0.17771739130434783,
 'swift': 0.175,
 'xml': 0.03260869565217391}

In [162]:
len(test_data)

459

In [17]:
def testbayes(testdata,plprob,plwordprob):
    Ypred = []

    for row in testdata:
        testcounter = Counter()
        testcounter.update(filter(None, re.split(r'[^\w]', re.sub(re.compile("/\*.*?\*/",re.DOTALL ) ,"" ,str(row[0])))))

        prob = {}
        for key in plprob.iterkeys():
            prob[key] = 0
        for key in prob.iterkeys():
            for i in testcounter:
                if i not in plwordprobs[key]:
                    plwordprob[key][i] = 1e-4
                else:
                    plwordprob[key][i] += 1e-4
                prob[key] += testcounter[i]*np.log(plwordprob[key][i])
            prob[key] += np.log(plprob[key])
        Ypred.append(max(prob.iteritems(), key=operator.itemgetter(1))[0])
    
    return Ypred

In [20]:
predictions = testbayes(test_data, plprobs, plwordprobs)

In [104]:
classifier_id = "pynotpy-natural-la-1510788551714"
natural_language_classifier = NaturalLanguageClassifierV1(
    username=username,
    password=password)

In [117]:
with open('trainingdatanew.csv', 'rb') as training_data:
    print(json.dumps(natural_language_classifier.create(training_data=training_data, name='pydata'), indent=2))

{
  "status": "Training", 
  "name": "pydata", 
  "language": "en", 
  "created": "2017-11-26T05:28:14.097Z", 
  "url": "https://gateway.watsonplatform.net/natural-language-classifier/api/v1/classifiers/ee2ec4x254-nlc-4275", 
  "status_description": "The classifier instance is in its training phase, not yet ready to accept classify requests", 
  "classifier_id": "ee2ec4x254-nlc-4275"
}


In [118]:
natural_language_classifier.status("ee2ec4x254-nlc-4275")

{u'classifier_id': u'ee2ec4x254-nlc-4275',
 u'created': u'2017-11-26T05:28:14.097Z',
 u'language': u'en',
 u'name': u'pydata',
 u'status': u'Training',
 u'status_description': u'The classifier instance is in its training phase, not yet ready to accept classify requests',
 u'url': u'https://gateway.watsonplatform.net/natural-language-classifier/api/v1/classifiers/ee2ec4x254-nlc-4275'}

In [154]:
natural_language_classifier.status("ee2ec4x254-nlc-4275")

{u'classifier_id': u'ee2ec4x254-nlc-4275',
 u'created': u'2017-11-26T05:28:14.097Z',
 u'language': u'en',
 u'name': u'pydata',
 u'status': u'Available',
 u'status_description': u'The classifier instance is now available and is ready to take classifier requests.',
 u'url': u'https://gateway.watsonplatform.net/natural-language-classifier/api/v1/classifiers/ee2ec4x254-nlc-4275'}

In [35]:
def compute_my_accuracy(pred, testdata):
    count = 0
    for i in range(len(pred)):
        if pred[i]==testdata[i][1]:
            count += 1
    return float(count)/len(pred)

In [129]:
def compute_watson_accuracy(testdata, classifier_id):
    count = 0
    for i in testdata:
        x = natural_language_classifier.classify(classifier_id,re.sub(' +',' '," ".join(re.split(r'[^\w]', re.sub(re.compile("/\*.*?\*/",re.DOTALL ) ,"" ,i[0]))))[0:1024])
        if x['top_class'] == i[1]:
            count += 1
    return float(count)/len(testdata)

watsonpred = []
for i in test_data:
    x = natural_language_classifier.classify("ee2ec4x254-nlc-4275",re.sub(' +',' '," ".join(re.split(r'[^\w]', re.sub(re.compile("/\*.*?\*/",re.DOTALL ) ,"" ,i[0]))))[0:1024])
    watsonpred.append(x['top_class']) 
    
def compute_watson_accuracy_fast(pred, testdata):
    count = 0
    for i in range(len(pred)):
        if pred[i]==testdata[i][1]:
            count += 1
    return float(count)/len(pred)
    
    

In [155]:
compute_my_accuracy(predictions, test_data)

0.971677559912854

In [156]:
compute_watson_accuracy_fast(watsonpred, test_data)

1.0

In [157]:
for i in range(len(test_data)):
    print test_data[i][1], watsonpred[i]

js js
swift swift
sh sh
swift swift
java java
m m
m m
m m
js js
sh sh
py py
java java
java java
sh sh
sh sh
java java
java java
swift swift
sh sh
js js
swift swift
swift swift
sh sh
java java
js js
py py
swift swift
sh sh
java java
js js
swift swift
sh sh
js js
java java
xml xml
sh sh
swift swift
swift swift
m m
js js
js js
js js
swift swift
java java
js js
js js
py py
java java
sh sh
go go
java java
sh sh
js js
m m
m m
js js
m m
go go
js js
js js
swift swift
sh sh
swift swift
sh sh
swift swift
swift swift
js js
java java
py py
java java
m m
m m
java java
js js
sh sh
swift swift
swift swift
go go
swift swift
java java
js js
sh sh
xml xml
js js
swift swift
swift swift
js js
sh sh
swift swift
sh sh
sh sh
js js
java java
sh sh
swift swift
m m
py py
js js
sh sh
go go
go go
swift swift
java java
sh sh
java java
java java
swift swift
sh sh
swift swift
sh sh
java java
js js
swift swift
sh sh
py py
swift swift
m m
js js
xml xml
go go
java java
m m
js js
sh sh
js js
js js
js js
js js
swift swif

In [112]:
backup = re.sub(' +',' '," ".join(re.split(r'[^\w]', re.sub(re.compile("/\*.*?\*/",re.DOTALL ) ,"" ,test_data[0][0]))))
backup_example = natural_language_classifier.classify("e552ebx250-nlc-641",backup)
backup_accuracy = compute_watson_accuracy(test_data,"e552ebx250-nlc-641")

In [120]:
introtext = 'var attachment = {"type":"image","payload":{"url":"https://pbs.twimg.com/profile_images/803642201653858305/IAW1DBPw_400x400.png","is_reusable": true}};controller.api.attachment_upload.upload(attachment, function (err, attachmentId) {if(err) {// Error} else {var image = {"attachment":{"type":"image","payload": {"attachment_id": attachmentId}}};bot.reply(message, image);}});'
introlist = [introtext]

In [158]:
print introtext

var attachment = {"type":"image","payload":{"url":"https://pbs.twimg.com/profile_images/803642201653858305/IAW1DBPw_400x400.png","is_reusable": true}};controller.api.attachment_upload.upload(attachment, function (err, attachmentId) {if(err) {// Error} else {var image = {"attachment":{"type":"image","payload": {"attachment_id": attachmentId}}};bot.reply(message, image);}});


In [159]:
natural_language_classifier.classify("e552ebx250-nlc-641", introtext)

{u'classes': [{u'class_name': u'js', u'confidence': 0.9847023377593169},
  {u'class_name': u'sh', u'confidence': 0.005604587315889725},
  {u'class_name': u'swift', u'confidence': 0.0034819667367551803},
  {u'class_name': u'go', u'confidence': 0.0020710387794202543},
  {u'class_name': u'py', u'confidence': 0.0012318330271757884},
  {u'class_name': u'xml', u'confidence': 0.0010572930320218526},
  {u'class_name': u'm', u'confidence': 0.0009756355454493819},
  {u'class_name': u'java', u'confidence': 0.0008753078039708989}],
 u'classifier_id': u'e552ebx250-nlc-641',
 u'text': u'var attachment = {"type":"image","payload":{"url":"https://pbs.twimg.com/profile_images/803642201653858305/IAW1DBPw_400x400.png","is_reusable": true}};controller.api.attachment_upload.upload(attachment, function (err, attachmentId) {if(err) {// Error} else {var image = {"attachment":{"type":"image","payload": {"attachment_id": attachmentId}}};bot.reply(message, image);}});',
 u'top_class': u'js',
 u'url': u'https://g

In [160]:
testbayes(introlist, plprobs, plwordprobs)

['js']

In [148]:
username="c01269a1-fe44-49be-94ba-6f257c263911"
password="Xe2LNum2uHwM"

In [164]:
len(train_data)

1840